# В этом ноутбуке представлено решение c catboost
*Оглавление:*
1. Полный пайнлайн с катбустом: катбуст и памятка по параметрам
2. Подбор гипов
3. Идеи для фичей под разные области

In [ ]:
!pip install catboost

# Стартовый пайплайн

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import numpy as np

# Полный пайплайн с катбусом

In [1]:
!pip install catboost

  Using cached catboost-1.2.8-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached matplotlib-3.10.7-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached plotly-6.4.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp310-cp310-manylinux2014_x86_6

In [5]:
pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [6]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

In [ ]:
#РЕГРЕССИЯ
data_path = 'вставить'
target = 'вставить'
# 1. Загрузка данных
data = pd.read_csv(data_path)

text_cols = data.select_dtypes(include=["object"]).columns.tolist()

safe_cat = []
remove_text = []

for col in text_cols:
    if data[col].str.len().mean() > 50:   # длинные тексты → удаляем
        remove_text.append(col)
    else:
        safe_cat.append(col)

print("Категориальные признаки:", safe_cat)
print("Удаляем текстовые признаки:", remove_text)

data = data.drop(columns=remove_text)

cat_features = safe_cat

# 2. Разделение признаков и целевой переменной
X = data.drop(target, axis=1)
y = data[target]

# 3. Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# 5. Определение модели
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    eval_metric='RMSE',
    verbose=100
)

# 6. Обучение
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# 7. Оценка
preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("RMSE:", mse ** 0.5)

# 8. Сохранение
model.save_model("catboost_regressor.cbm")
preds = model.predict(X_test)

submission = pd.DataFrame({
    "id": test["id"],
    "target": preds
})
submission.to_csv("submission_reg.csv", index=False)


In [7]:
#БИНАРНАЯ КЛАССИФИКАЦИЯ
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from catboost import CatBoostClassifier, Pool

# ===========================
# 1. Загрузка данных
# ===========================

data_path = 'вставить'
target = 'вставить'

data = pd.read_csv(data_path)

# Выделяем текстовые столбцы
text_cols = data.select_dtypes(include=["object"]).columns.tolist()

safe_cat = []
remove_text = []

for col in text_cols:
    if data[col].str.len().mean() > 50:   # длинные тексты → удаляем
        remove_text.append(col)
    else:
        safe_cat.append(col)

print("Категориальные признаки:", safe_cat)
print("Удаляем текстовые признаки:", remove_text)

# Удаляем длинные текстовые признаки
data = data.drop(columns=remove_text)

cat_features = safe_cat

# ===========================
# 2. Разделение признаков/цели
# ===========================

X = data.drop(target, axis=1)
y = data[target]

# ===========================
# 3. Train/Test Split
# ===========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# ===========================
# 4. Модель (CatBoostClassifier)
# ===========================

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',   # бинарная классификация
    eval_metric='AUC',         # можно 'Accuracy'
    verbose=100
)

# ===========================
# 5. Обучение
# ===========================

model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# ===========================
# 6. Оценка
# ===========================

preds_proba = model.predict_proba(X_test)[:, 1]   # вероятности
preds = (preds_proba >= 0.5).astype(int)          # классы

acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, preds_proba)

print("Accuracy:", acc)
print("AUC:", auc)

# ===========================
# 7. Сохранение модели
# ===========================

model.save_model("catboost_binary_classifier.cbm")

# ===========================
# 8. Сабмит
# ===========================

# Пример, если есть test.csv:
# submission = pd.DataFrame({
#     "id": test["id"],
#     "target": model.predict_proba(test)[:, 1]
# })
# submission.to_csv("submission_binary.csv", index=False)


In [8]:
#МУЛЬТИКЛАССОВАЯ КЛАССИФИКАЦИЯ
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool

# ===========================
# 1. Загрузка данных
# ===========================

data_path = 'вставить'
target = 'вставить'

data = pd.read_csv(data_path)

# Выделяем текстовые столбцы
text_cols = data.select_dtypes(include=["object"]).columns.tolist()

safe_cat = []
remove_text = []

for col in text_cols:
    if data[col].str.len().mean() > 50:   # длинные тексты → удаляем
        remove_text.append(col)
    else:
        safe_cat.append(col)

print("Категориальные признаки:", safe_cat)
print("Удаляем текстовые признаки:", remove_text)

# Удаляем длинные текстовые признаки
data = data.drop(columns=remove_text)

cat_features = safe_cat

# ===========================
# 2. Разделение признаков/цели
# ===========================

X = data.drop(target, axis=1)
y = data[target]

# ===========================
# 3. Train/Test Split
# ===========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# ===========================
# 4. Модель (CatBoostClassifier)
# ===========================

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiClass',   # многоклассовая классификация
    eval_metric='MultiClass',
    verbose=100
)

# ===========================
# 5. Обучение
# ===========================

model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# ===========================
# 6. Оценка
# ===========================

preds = model.predict(X_test)          # предсказанные классы
preds = preds.reshape(-1)              # CatBoost даёт столбец — распрямляем

acc = accuracy_score(y_test, preds)

print("Accuracy:", acc)

# ===========================
# 7. Сохранение модели
# ===========================

model.save_model("catboost_multiclass.cbm")

# ===========================
# 8. Сабмит
# ===========================

# Пример, если есть test.csv:
# submission = pd.DataFrame({
#     "id": test["id"],
#     "target": model.predict(test)
# })
# submission.to_csv("submission_multiclass.csv", index=False)


In [ ]:
#РАНЖИРОВАНИЕ
from catboost import CatBoostRanker, Pool
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRanker, Pool
from sklearn.metrics import ndcg_score

# ===========================
# 1. Загрузка данных
# ===========================

data_path = 'вставить'
target = 'вставить'        # релевантность
group_col = 'group_id'     # ГЛАВНОЕ поле для ранжирования

data = pd.read_csv(data_path)

# ===========================
# 1.1. Обработка текстовых колонок
# ===========================

text_cols = data.select_dtypes(include=["object"]).columns.tolist()

safe_cat = []
remove_text = []

for col in text_cols:
    if data[col].str.len().mean() > 50:   # длинные тексты → удаляем
        remove_text.append(col)
    else:
        safe_cat.append(col)

print("Категориальные признаки:", safe_cat)
print("Удаляем текстовые:", remove_text)

data = data.drop(columns=remove_text)

cat_features = safe_cat

# ===========================
# 2. Разделение признаков и цели
# ===========================

X = data.drop([target], axis=1)
y = data[target]

# ===========================
# 3. Train/Test Split
# ===========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X[group_col]
)

# Важно: CatBoost требует массив размера групп
train_group = X_train[group_col].values
test_group = X_test[group_col].values

# Удаляем group_id из признаков (он не должен быть фичей)
X_train = X_train.drop(columns=[group_col])
X_test = X_test.drop(columns=[group_col])

train_pool = Pool(
    X_train, y_train,
    group_id=train_group,
    cat_features=cat_features
)

test_pool = Pool(
    X_test, y_test,
    group_id=test_group,
    cat_features=cat_features
)

# ===========================
# 4. Модель CatBoostRanker
# ===========================

model = CatBoostRanker(
    iterations=1500,
    learning_rate=0.05,
    depth=8,
    loss_function='YetiRank',   # лучший LTR лосс
    eval_metric='NDCG:top=10',
    verbose=100
)

# ===========================
# 5. Обучение
# ===========================

model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# ===========================
# 6. Оценка
# ===========================

preds = model.predict(X_test)

# Группируем ответы по group_id
test_df = X_test.copy()
test_df["target"] = y_test
test_df["pred"] = preds
test_df["group"] = test_group

ndcgs = []
for g, df_g in test_df.groupby("group"):
    if df_g["target"].nunique() > 1:
        ndcgs.append(
            ndcg_score([df_g["target"].values], [df_g["pred"].values], k=10)
        )

print("Средний NDCG@10:", sum(ndcgs) / len(ndcgs))

# ===========================
# 7. Сохранение модели
# ===========================

model.save_model("catboost_ranker.cbm")

# ===========================
# 8. Сабмит (пример)
# ===========================

# submission = pd.DataFrame({
#     "group_id": test_group,
#     "score": preds
# })
# submission.to_csv("submission_rank.csv", index=False)


Гиперпараметры:
- Чем менее глубокая модель, тем больше iterations

iterations и learning_rate всегда идут в паре:
- меньше learning_rate → нужно больше iterations.
- больше learning_rate → нужно меньше iterations

Когда увеличить (depth=8–10):
- Данные сложные, нелинейные зависимости, много признаков.
Набор данных большой → риск переобучения меньше.
Когда уменьшить (depth=4–6):
- Мало данных.
- Признаки простые или уже хорошо обработаны (например, после feature engineering).
- Переобучение.

l2_leaf_reg — L2-регуляризация (штраф на сложность модели)
Что делает:
Сглаживает веса, помогает бороться с переобучением.
Когда увеличить (например, 10, 20, 50):
- Модель переобучается.
- Набор данных небольшой или шумный.
Когда уменьшить (например, 1, 2, 3):
-Модель недообучается.
- Данные большие и "чистые".

bagging_temperature — степень случайности (аналог subsample)
Что делает:
Добавляет стохастичность при выборе объектов для каждой итерации.
Когда увеличить (1–2):
- Переобучение → добавляем больше случайности.
- Улучшает обобщающую способность.
Когда уменьшить (0–0.5):
- Недообучение → уменьшаем рандомизацию, делаем обучение стабильнее.

grow_policy
'SymmetricTree' — стандартная стратегия (сбалансированные деревья, быстро).
'Depthwise' или 'Lossguide' — гибче, может работать лучше на больших данных с большим количеством признаков, но медленнее.
Когда менять:
- 'SymmetricTree' хорошо для большинства задач.
- 'Lossguide' полезен, если признаков > 1000.

Примеры практических ситуаций
Ситуация	
Модель переобучается	↓ depth, ↓ learning_rate, ↑ l2_leaf_reg, ↑ bagging_temperature
Модель недообучается	↑ depth, ↑ iterations, ↓ l2_leaf_reg
Модель очень медленно обучается	↑ learning_rate, ↓ iterations, ↓ depth
Много шумных данных	↑ l2_leaf_reg, ↓ depth, ↑ bagging_temperature
Много категориальных признаков	↓ one_hot_max_size, ↑ l2_leaf_reg

# Подбор гипов

In [ ]:
# --Grid search--
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

model = CatBoostClassifier(
    iterations=500,
    cat_features=cat_features,
    verbose=0
)

param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 10],
    'bagging_temperature': [0, 1, 2]
}

grid = GridSearchCV(model, param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
grid.fit(X, y)
print("Лучшие параметры:", grid.best_params_)

# --Randomized Search--
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from scipy.stats import randint, uniform

model = CatBoostClassifier(
    iterations=1000,
    cat_features=cat_features,
    verbose=0
)

param_dist = {
    'depth': randint(4, 10),
    'learning_rate': uniform(0.01, 0.1),
    'l2_leaf_reg': uniform(1, 10),
    'bagging_temperature': uniform(0, 2),
}

search = RandomizedSearchCV(
    model, 
    param_distributions=param_dist,
    n_iter=30,  # количество случайных комбинаций
    scoring='roc_auc',
    cv=3,
    random_state=42,
    n_jobs=-1
)

search.fit(X, y)
print("Лучшие параметры:", search.best_params_)

# --Grid search в CatBoost
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=500,
    loss_function='Logloss',
    eval_metric='AUC',
    cat_features=cat_features,
    verbose=False
)

grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 10]
}

grid_result = model.grid_search(grid, X=X, y=y, cv=3, shuffle=True, stratified=True)
print(grid_result['params'])

# --Optuna--
!pip install optuna
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 15, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 2),
        'random_strength': trial.suggest_float('random_strength', 0, 5),
        'border_count': trial.suggest_int('border_count', 32, 256),
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'verbose': 0
    }

    model = CatBoostClassifier(**params)
    score = cross_val_score(model, X, y, cv=3, scoring='roc_auc', n_jobs=-1)
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Лучшие параметры:", study.best_params)


In [ ]:
# --Early stopping--
model = CatBoostClassifier(
    iterations=2000,              # максимальное количество итераций
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',
    eval_metric='AUC',
    od_type='Iter',               # тип остановки (по числу итераций)
    od_wait=50,                   # если 50 итераций без улучшения — стоп
    random_seed=42,
    verbose=100
)
# Передаём eval_set (валидацию)
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

In [ ]:
# --Cross Validation--
from sklearn.model_selection import cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',
    cat_features=cat_features,
    verbose=0
)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold, scoring='roc_auc', n_jobs=-1)

print("AUC по фолдам:", scores)
print("Средний AUC:", scores.mean())

# регрессия
from catboost import CatBoostRegressor, Pool, cv

params = {
    'iterations': 1500,
    'learning_rate': 0.03,
    'depth': 6,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'od_type': 'Iter',
    'od_wait': 50,
    'verbose': 100
}

train_pool = Pool(X, y, cat_features=cat_features)

cv_data = cv(
    pool=train_pool,
    params=params,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42
)

print("✅ Best RMSE:", cv_data["test-RMSE-mean"].min())
print("✅ Best iteration:", cv_data["test-RMSE-mean"].idxmin())

# Feature Engineering

In [ ]:
#работа с пропусками 
X = X.fillna(X.median(numeric_only=True))
#работа с числовыми признаками
#создание взаимодействий
#дата и время 
X["date"] = pd.to_datetime(X["date"])
X["year"] = X["date"].dt.year
X["month"] = X["date"].dt.month
X["day"] = X["date"].dt.day
X["dayofweek"] = X["date"].dt.dayofweek
X["is_weekend"] = X["dayofweek"].isin([5, 6]).astype(int)
X["hour"] = X["date"].dt.hour
X["month_sin"] = np.sin(2 * np.pi * X["month"]/12)
X["month_cos"] = np.cos(2 * np.pi * X["month"]/12)
#групповые агрегаты
grouped = X.groupby('customer_id')['amount'].agg(['mean', 'sum', 'std', 'count']).reset_index()
grouped.columns = ['customer_id', 'amount_mean', 'amount_sum', 'amount_std', 'amount_count']
X = X.merge(grouped, on='customer_id', how='left')
#отбор признаков

#объединение редких категорий
for col in cat_features:
    freqs = X[col].value_counts()
    rare = freqs[freqs < 50].index
    X[col] = X[col].replace(rare, 'rare')

# Идеи для фичей в узконаправленных задачах
    1. Кредитный скоринг / Банковские данные
Цель: предсказать дефолт / вероятность невозврата.
Идеи признаков:

income_to_loan_ratio — отношение дохода к сумме кредита
age_group — возрастные категории
employment_length — стаж
credit_utilization — текущие долги / кредитный лимит
debt_to_income — долговая нагрузка
previous_loans_count — количество прошлых кредитов
avg_payment_delay — средняя задержка платежей
num_open_accounts — активные счета
has_mortgage, has_car_loan — бинарные признаки


In [ ]:
X['income_to_loan'] = X['income'] / (X['loan_amount'] + 1)
X['debt_to_income'] = X['total_debt'] / (X['income'] + 1)
X['credit_utilization'] = X['used_credit'] / (X['credit_limit'] + 1)
X['employment_years'] = (pd.Timestamp('today').year - X['employment_start_year'])



    2. Географические признаки (гео-данные)
Применимо к доставкам, логистике, геоаналитике.
Идеи признаков:

distance — гео-дистанция (Haversine formula)
region, city, postal_code
distance_to_center
avg_order_distance_by_user

In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # км
    d_lat = np.radians(lat2 - lat1)
    d_lon = np.radians(lon2 - lon1)
    a = np.sin(d_lat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(d_lon / 2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

X["distance_km"] = haversine(X["pickup_lat"], X["pickup_lon"], X["drop_lat"], X["drop_lon"])


    3. Временные ряды
Для прогнозирования спроса, продаж, трафика, кликов и т.д.
Идеи признаков:

лаги: y(t-1), y(t-7)
скользящее среднее: rolling_mean, rolling_std
тренд (diff, pct_change)
праздничные дни, сезонность

In [ ]:
X["lag_1"] = X["target"].shift(1)
X["lag_7"] = X["target"].shift(7)
X["rolling_mean_7"] = X["target"].shift(1).rolling(7).mean()
X["rolling_std_7"] = X["target"].shift(1).rolling(7).std()
X["diff"] = X["target"].diff()

    4. E-commerce / Retail
Цель: предсказать покупки, выручку, отток клиента.
Идеи признаков:

avg_purchase_value
total_orders, unique_products
days_since_last_purchase
avg_time_between_purchases
is_weekend_purchase

In [ ]:
X['days_since_last_purchase'] = (X['current_date'] - X['last_purchase_date']).dt.days
X['avg_order_value'] = X['total_spent'] / (X['orders_count'] + 1)

    5. Логистика / Транспорт
Время прибытия, маршруты, задержки.
Идеи признаков:

route_length_km
avg_speed = distance / duration
delay = actual_arrival - scheduled_arrival
is_rush_hour, is_weekend

In [ ]:
X["avg_speed"] = X["distance_km"] / (X["duration_min"]/60)
X["delay_min"] = (X["actual_arrival"] - X["scheduled_arrival"]).dt.total_seconds() / 60

    6. Базовые текстовые статистики

In [ ]:
import pandas as pd
import numpy as np
import re

def basic_text_features(df, text_col):
    df[f"{text_col}_len"] = df[text_col].astype(str).apply(len)
    df[f"{text_col}_word_count"] = df[text_col].astype(str).apply(lambda x: len(x.split()))
    df[f"{text_col}_unique_words"] = df[text_col].astype(str).apply(lambda x: len(set(x.split())))
    df[f"{text_col}_punct_count"] = df[text_col].astype(str).apply(lambda x: len(re.findall(r'[^\w\s]', x)))
    df[f"{text_col}_caps_ratio"] = df[text_col].astype(str).apply(
        lambda x: sum(1 for c in x if c.isupper()) / (len(x) + 1)
    )
    df[f"{text_col}_avg_word_len"] = df[text_col].astype(str).apply(
        lambda x: np.mean([len(w) for w in x.split()]) if len(x.split()) > 0 else 0
    )
    return df

    7. Признаки на основе эмоциональной окраски текста

In [ ]:
pip install textblob
from textblob import TextBlob

def sentiment_features(df, text_col):
    df[f"{text_col}_polarity"] = df[text_col].astype(str).apply(lambda x: TextBlob(x).sentiment.polarity)
    df[f"{text_col}_subjectivity"] = df[text_col].astype(str).apply(lambda x: TextBlob(x).sentiment.subjectivity)
    return df


    8. Тематические фичи (topic modeling — LDA)
Такие фичи хорошо отражают "о чём текст", особенно для длинных документов.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def topic_features(df, text_col, n_topics=5, max_features=1000):
    cv = CountVectorizer(max_features=max_features, stop_words='english')
    bow = cv.fit_transform(df[text_col].fillna(''))
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    topics = lda.fit_transform(bow)
    topic_df = pd.DataFrame(topics, columns=[f"topic_{i}" for i in range(n_topics)])
    return pd.concat([df.reset_index(drop=True), topic_df.reset_index(drop=True)], axis=1)

    9. Идеи текстовых фич по типам задач

Тип задачи	                       Полезные текстовые признаки
Отзывы (Sentiment)	               длина текста, polarity, subjectivity, TF-IDF, частоты слов “good”, “bad”
Технические тикеты / support	   TF-IDF, частоты слов “error”, “fail”, “crash”, длина, количество !, ?
E-commerce описания	               длина, уникальные слова, частоты “new”, “sale”, “discount”, topic features
Новости / статьи	               LDA темы, TF-IDF, sentence embeddings
Резюме / вакансии	               частоты по профессиям, длина текста, topic modeling (“skills”, “experience”)
Юридические тексты	               количество пунктов, длина, доля чисел, topic modeling по терминам
Социальные сети / комментарии	   polarity, caps ratio, emoji count, sentiment, avg word length

In [ ]:
#Отзывы (Sentiment)
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

# ==== Вспомогательные функции ====
def safe_text(x):
    return str(x) if not pd.isna(x) else ""

def word_tokens(text):
    return re.findall(r"\b[\w']+\b", safe_text(text).lower())

def lexicon_sentiment(text):
    POS = {"good", "great", "excellent", "love", "best", "nice", "amazing"}
    NEG = {"bad", "terrible", "awful", "hate", "poor", "worst"}
    SUBJ = {"think", "feel", "believe", "maybe", "might", "seems"}
    toks = word_tokens(text)
    if not toks:
        return 0, 0
    pos = sum(t in POS for t in toks)
    neg = sum(t in NEG for t in toks)
    subj = sum(t in SUBJ for t in toks)
    return (pos - neg) / len(toks), subj / len(toks)

# ==== Трансформер для отзывов ====
class ReviewTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", tfidf_max_features=300):
        self.text_col = text_col
        self.tfidf_max_features = tfidf_max_features
        self.tfidf_vec = None

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        self.tfidf_vec = TfidfVectorizer(max_features=self.tfidf_max_features, stop_words="english")
        self.tfidf_vec.fit(texts)
        return self

    def transform(self, X):
        df = X.copy()
        texts = df[self.text_col].fillna("").astype(str).tolist()
        feats = {}

        feats["len"] = [len(t) for t in texts]
        polsub = [lexicon_sentiment(t) for t in texts]
        feats["polarity"] = [p for p, s in polsub]
        feats["subjectivity"] = [s for p, s in polsub]
        feats["freq_good"] = [t.lower().count("good") for t in texts]
        feats["freq_bad"] = [t.lower().count("bad") for t in texts]

        tfidf = self.tfidf_vec.transform(texts)
        tfidf_df = pd.DataFrame(
            tfidf.toarray(),
            columns=[f"tfidf_{w}" for w in self.tfidf_vec.get_feature_names_out()]
        )

        feat_df = pd.concat([pd.DataFrame(feats), tfidf_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)


In [ ]:
# Технические тикеты / support	   TF-IDF, частоты слов “error”, “fail”, “crash”, длина, количество !, ?
class SupportTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", tfidf_max_features=300):
        self.text_col = text_col
        self.tfidf_max_features = tfidf_max_features
        self.tfidf_vec = None

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        self.tfidf_vec = TfidfVectorizer(max_features=self.tfidf_max_features, stop_words="english")
        self.tfidf_vec.fit(texts)
        return self

    def transform(self, X):
        df = X.copy()
        texts = df[self.text_col].fillna("").astype(str).tolist()
        feats = {}

        feats["len"] = [len(t) for t in texts]
        feats["excl_count"] = [t.count("!") for t in texts]
        feats["quest_count"] = [t.count("?") for t in texts]
        feats["freq_error"] = [t.lower().count("error") for t in texts]
        feats["freq_fail"] = [t.lower().count("fail") for t in texts]
        feats["freq_crash"] = [t.lower().count("crash") for t in texts]

        tfidf = self.tfidf_vec.transform(texts)
        tfidf_df = pd.DataFrame(
            tfidf.toarray(),
            columns=[f"tfidf_{w}" for w in self.tfidf_vec.get_feature_names_out()]
        )

        feat_df = pd.concat([pd.DataFrame(feats), tfidf_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)

In [ ]:
#Применение
# Пример данных
reviews_df = pd.DataFrame({
    "text": [
        "I love this product, it's really good and amazing!",
        "Terrible experience, bad quality and poor support.",
        "Good price but not the best design."
    ]
})

support_df = pd.DataFrame({
    "text": [
        "App crash after update! Please fix this error asap!",
        "Getting fail message when trying to login??",
        "Error 404 on page load"
    ]
})

# --- Отзывы ---
rev_trans = ReviewTextFeatures(text_col="text")
rev_features = rev_trans.fit_transform(reviews_df)
print("Отзывы (Sentiment):")
print(rev_features.head())

# --- Support тикеты ---
sup_trans = SupportTextFeatures(text_col="text")
sup_features = sup_trans.fit_transform(support_df)
print("\nТехнические тикеты:")
print(sup_features.head())


In [ ]:
#E-commerce описания
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# ===== Вспомогательные функции =====
def safe_text(x):
    return str(x) if not pd.isna(x) else ""

def unique_word_count(text):
    words = re.findall(r"\b[a-zA-Z]+\b", safe_text(text).lower())
    return len(set(words))

# ==========================================
# 🛍️ E-commerce текстовые признаки
# ==========================================
class EcommerceTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", lda_topics=5, max_features=500):
        self.text_col = text_col
        self.lda_topics = lda_topics
        self.max_features = max_features
        self.count_vec = None
        self.lda = None
        self.target_words = ["new", "sale", "discount"]

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        self.count_vec = CountVectorizer(max_features=self.max_features, stop_words="english")
        bow = self.count_vec.fit_transform(texts)
        self.lda = LatentDirichletAllocation(n_components=self.lda_topics, random_state=42)
        self.lda.fit(bow)
        return self

    def transform(self, X):
        texts = X[self.text_col].fillna("").astype(str).tolist()

        feats = {}
        feats["len"] = [len(t) for t in texts]
        feats["unique_words"] = [unique_word_count(t) for t in texts]
        for w in self.target_words:
            feats[f"freq_{w}"] = [t.lower().count(w) for t in texts]

        bow = self.count_vec.transform(texts)
        topics = self.lda.transform(bow)
        topic_df = pd.DataFrame(topics, columns=[f"topic_{i}" for i in range(topics.shape[1])])

        feat_df = pd.concat([pd.DataFrame(feats), topic_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)


In [ ]:
#Применение
# ==== Пример данных ====
ecom_df = pd.DataFrame({
    "text": [
        "New stylish summer dress on sale! Get a 20% discount now.",
        "Discount on new arrivals: men's jackets and coats available online.",
        "Buy our best-selling shoes — elegant, comfortable, and new design.",
        "Exclusive sale! Modern laptop bag with sleek style and functionality."
    ]
})

# --- E-commerce ---
ecom_trans = EcommerceTextFeatures(text_col="text")
ecom_features = ecom_trans.fit_transform(ecom_df)

print("🛍️ E-commerce текстовые признаки:")
print(ecom_features.head())


In [ ]:
#Новости / статьи	  
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

def safe_text(x):
    return str(x) if not pd.isna(x) else ""

class NewsTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", tfidf_max_features=500, embed_dim=16, lda_topics=5):
        self.text_col = text_col
        self.tfidf_max_features = tfidf_max_features
        self.embed_dim = embed_dim
        self.lda_topics = lda_topics

        self.tfidf_vec = None
        self.svd = None
        self.count_vec = None
        self.lda = None

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        # TF-IDF
        self.tfidf_vec = TfidfVectorizer(max_features=self.tfidf_max_features, stop_words="english")
        tfidf_mat = self.tfidf_vec.fit_transform(texts)

        # Sentence embeddings через SVD
        dim = min(self.embed_dim, tfidf_mat.shape[1]-1 if tfidf_mat.shape[1] > 1 else 1)
        self.svd = TruncatedSVD(n_components=dim, random_state=42)
        self.svd.fit(tfidf_mat)

        # LDA по Bag of Words
        self.count_vec = CountVectorizer(max_features=self.tfidf_max_features, stop_words="english")
        bow = self.count_vec.fit_transform(texts)
        self.lda = LatentDirichletAllocation(n_components=self.lda_topics, random_state=42)
        self.lda.fit(bow)
        return self

    def transform(self, X):
        texts = X[self.text_col].fillna("").astype(str).tolist()

        # TF-IDF
        tfidf = self.tfidf_vec.transform(texts)
        tfidf_df = pd.DataFrame(tfidf.toarray(), columns=[f"tfidf_{w}" for w in self.tfidf_vec.get_feature_names_out()])

        # Sentence embeddings
        emb = self.svd.transform(tfidf)
        emb_df = pd.DataFrame(emb, columns=[f"embed_{i}" for i in range(emb.shape[1])])

        # LDA topics
        bow = self.count_vec.transform(texts)
        topics = self.lda.transform(bow)
        topic_df = pd.DataFrame(topics, columns=[f"topic_{i}" for i in range(topics.shape[1])])

        # Объединение
        feat_df = pd.concat([tfidf_df, emb_df, topic_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)

In [ ]:
# Резюме / вакансии	           
class ResumeTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", tfidf_max_features=300, lda_topics=4):
        self.text_col = text_col
        self.tfidf_max_features = tfidf_max_features
        self.lda_topics = lda_topics

        self.count_vec = None
        self.lda = None

        self.professions = ["engineer", "developer", "manager", "analyst", "designer", "consultant"]
        self.skills = ["python", "sql", "excel", "communication", "leadership", "project", "experience"]

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        self.count_vec = CountVectorizer(max_features=self.tfidf_max_features, stop_words="english")
        bow = self.count_vec.fit_transform(texts)
        self.lda = LatentDirichletAllocation(n_components=self.lda_topics, random_state=42)
        self.lda.fit(bow)
        return self

    def transform(self, X):
        texts = X[self.text_col].fillna("").astype(str).tolist()

        feats = {}
        feats["len"] = [len(t) for t in texts]
        feats["prof_count"] = [sum(t.lower().count(p) for p in self.professions) for t in texts]
        feats["skills_count"] = [sum(t.lower().count(s) for s in self.skills) for t in texts]

        # Topic modeling
        bow = self.count_vec.transform(texts)
        topics = self.lda.transform(bow)
        topic_df = pd.DataFrame(topics, columns=[f"topic_{i}" for i in range(topics.shape[1])])

        feat_df = pd.concat([pd.DataFrame(feats), topic_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)
        
    

In [ ]:
# --ПРИМЕНЕНИЕ--
    # ==== Пример данных ====
news_df = pd.DataFrame({
    "text": [
        "The government passed a new economic reform bill today.",
        "Stock markets saw a sharp decline after the inflation report.",
        "Scientists discovered a new planet similar to Earth.",
        "The local football team won the championship last night."
    ]
})

resume_df = pd.DataFrame({
    "text": [
        "Experienced software engineer with strong Python and SQL skills. Managed multiple projects.",
        "Marketing manager with 10 years of leadership experience and strong communication skills.",
        "Junior data analyst skilled in Excel and data visualization.",
        "Project designer experienced in architecture and planning."
    ]
})

# --- Новости ---
news_trans = NewsTextFeatures(text_col="text")
news_features = news_trans.fit_transform(news_df)
print("Новости / статьи:")
print(news_features.head())

# --- Резюме ---
resume_trans = ResumeTextFeatures(text_col="text")
resume_features = resume_trans.fit_transform(resume_df)
print("\nРезюме / вакансии:")
print(resume_features.head())

In [ ]:
#Юридические тексты
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# ======= Вспомогательные функции =======
def safe_text(x):
    return str(x) if not pd.isna(x) else ""

def word_tokens(text):
    return re.findall(r"\b[\w']+\b", safe_text(text).lower())

def caps_ratio(text):
    s = safe_text(text)
    if not s:
        return 0
    return sum(1 for c in s if c.isupper()) / (len(s) + 1e-9)

def emoji_count(text):
    return sum(1 for c in safe_text(text) if ord(c) in range(0x1F300, 0x1F9FF))

def avg_word_len(text):
    toks = word_tokens(text)
    return np.mean([len(t) for t in toks]) if toks else 0

def digits_fraction(text):
    s = safe_text(text)
    return sum(1 for c in s if c.isdigit()) / (len(s) + 1e-9)

def lexicon_sentiment(text):
    POS = {"good", "great", "excellent", "love", "best", "nice", "amazing"}
    NEG = {"bad", "terrible", "awful", "hate", "poor", "worst"}
    toks = word_tokens(text)
    if not toks:
        return 0
    score = sum((1 if t in POS else -1 if t in NEG else 0) for t in toks)
    return score / len(toks)

# ==========================================
# ⚖️ Юридические тексты
# ==========================================
class LegalTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text", lda_topics=5, max_features=500):
        self.text_col = text_col
        self.lda_topics = lda_topics
        self.max_features = max_features

        self.count_vec = None
        self.lda = None
        self.legal_terms = [
            "agreement", "contract", "party", "clause", "warranty",
            "liability", "condition", "term", "obligation", "notice"
        ]

    def fit(self, X, y=None):
        texts = X[self.text_col].fillna("").astype(str)
        self.count_vec = CountVectorizer(max_features=self.max_features, stop_words="english")
        bow = self.count_vec.fit_transform(texts)
        self.lda = LatentDirichletAllocation(n_components=self.lda_topics, random_state=42)
        self.lda.fit(bow)
        return self

    def transform(self, X):
        texts = X[self.text_col].fillna("").astype(str).tolist()
        feats = {}
        feats["len"] = [len(t) for t in texts]
        feats["num_points"] = [len(re.findall(r"\b\d+\.", t)) for t in texts]
        feats["digits_frac"] = [digits_fraction(t) for t in texts]
        feats["legal_term_count"] = [sum(t.lower().count(term) for term in self.legal_terms) for t in texts]

        bow = self.count_vec.transform(texts)
        topics = self.lda.transform(bow)
        topic_df = pd.DataFrame(topics, columns=[f"topic_{i}" for i in range(topics.shape[1])])

        feat_df = pd.concat([pd.DataFrame(feats), topic_df], axis=1)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)

In [ ]:
#Социальные сети / комментарии
class SocialTextFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_col="text"):
        self.text_col = text_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X[self.text_col].fillna("").astype(str).tolist()
        feats = {}
        feats["polarity"] = [lexicon_sentiment(t) for t in texts]
        feats["caps_ratio"] = [caps_ratio(t) for t in texts]
        feats["emoji_count"] = [emoji_count(t) for t in texts]
        feats["avg_word_len"] = [avg_word_len(t) for t in texts]
        feats["sentiment_score"] = [np.sign(lexicon_sentiment(t)) for t in texts]

        feat_df = pd.DataFrame(feats)
        feat_df.columns = [f"{self.text_col}_{c}" for c in feat_df.columns]
        return feat_df.reset_index(drop=True)

In [ ]:
#Применение
# ==== Пример данных ====
legal_df = pd.DataFrame({
    "text": [
        "1. This agreement is made between the two parties. 2. Each party has obligations under clause 5.",
        "The contract includes warranty terms and liability conditions.",
        "Parties agree to provide notice of termination in writing."
    ]
})

social_df = pd.DataFrame({
    "text": [
        "OMG I LOVE this 😍😍 AMAZING!!!",
        "bad service, hate it :((",
        "Wow GREAT job team!!! So proud 💪🔥"
    ]
})

# --- Юридические тексты ---
legal_trans = LegalTextFeatures(text_col="text")
legal_features = legal_trans.fit_transform(legal_df)
print("⚖️ Юридические тексты:")
print(legal_features.head())

# --- Социальные сети / комментарии ---
social_trans = SocialTextFeatures(text_col="text")
social_features = social_trans.fit_transform(social_df)
print("\n💬 Социальные сети / комментарии:")
print(social_features.head())
